# MobileViT-XS PyTorch Model - Quantization for IMX500

[Run this tutorial in Google Colab](https://colab.research.google.com/github/sony/model_optimization/blob/main/tutorials/notebooks/pytorch/ptq/pytorch_mobilevit_xs_for_imx500.ipynb)

## Overview

In this tutorial, we will illustrate a basic and quick process of preparing a pre-trained model for deployment using MCT. Specifically, we will demonstrate how to download a pre-trained MobileViT-XS model from the MCT Models Library, compress it, and make it deployment-ready using MCT's post-training quantization techniques.

We will use an existing pre-trained MobileViT-XS model based on [Timm](https://github.com/huggingface/pytorch-image-models). The model was slightly adjusted for model quantization. We will quantize the model using MCT post training quantization and evaluate the performance of the floating point model and the quantized model on ImageNet dataset.


## Summary

In this tutorial we will cover:

1. Post-Training Quantization using MCT of PyTorch classification model.
2. Data preparation - loading and preprocessing validation and representative datasets from ImageNet.
3. Accuracy evaluation of the floating-point and the quantized models.

## Setup
### Install the relevant packages

In [ ]:
!pip install -q torch
!pip install onnx
!pip install timm
!pip install 'huggingface-hub>=0.21.0'

 Clone a copy of the [MCT](https://github.com/sony/model_optimization) (Model Compression Toolkit) into your current directory. This step ensures that you have access to [MCT Models Garden](https://github.com/sony/model_optimization/tree/main/tutorials/mct_model_garden) folder which contains all the necessary utility functions for this tutorial.
  **It's important to note that we use the most up-to-date MCT code available.**

In [ ]:
!git clone https://github.com/sony/model_optimization.git local_mct
!pip install -r ./local_mct/requirements.txt
import sys
sys.path.insert(0,"./local_mct")

### Download ImageNet validation set

In [ ]:
!mkdir imagenet
!mkdir imagenet/val
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_val.tar
!mv ILSVRC2012_img_val.tar imagenet/
!tar -xvf imagenet/ILSVRC2012_img_val.tar -C imagenet/val
!echo Done loading ILSVRC2012_img_val images

## Model Quantization

### Download a Pre-Trained Model 

We begin by loading a pre-trained [MobileViT-XS](https://huggingface.co/SSI-DNN/pytorch_mobilevit_xs) model. This implementation is based on [Timm](https://github.com/huggingface/pytorch-image-models) and includes a slightly modified version of timm/Attention module that was adapted for model quantization. For further insights into the model's implementation details, please refer to [MCT Models Garden - yolov8](https://github.com/sony/model_optimization/tree/main/tutorials/mct_model_garden/models_pytorch/mobilevit_xs).  

In [ ]:
from tutorials.mct_model_garden.models_pytorch.mobilevit_xs.mobilevit_xs import MobileViTXSPyTorch
model = MobileViTXSPyTorch.from_pretrained("SSI-DNN/pytorch_mobilevit_xs")

### Post training quantization using Model Compression Toolkit 

Now, we're all set to use MCT's post-training quantization. To begin, we'll define a representative dataset and proceed with the model quantization. Please note that, for demonstration purposes, we'll use the evaluation dataset as our representative dataset. We'll calibrate the model using 80 representative images, divided into 20 iterations of 'batch_size' images each. 

Additionally, to further compress the model's memory footprint, we will employ the mixed-precision quantization technique. This method allows each layer to be quantized with different precision options: 2, 4, and 8 bits, aligning with the imx500 target platform capabilities.

In [ ]:
import model_compression_toolkit as mct
from timm.data import create_dataset, create_loader, resolve_data_config
from typing import Iterator, Tuple, List

DATASET = 'Imagenet'
REPRESENTATIVE_DATASET_FOLDER = './imagenet/val'
BATCH_SIZE = 4
n_iters = 20
IMG_SIZE = 256
DATA_ARGS = {'img_size': IMG_SIZE}

# Load representative dataset
data_config = resolve_data_config(args=DATA_ARGS,
                                  model=model)

val_dataset = create_dataset(name=DATASET, root=REPRESENTATIVE_DATASET_FOLDER, is_training=False,
                             batch_size=BATCH_SIZE)
representative_dataset = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=BATCH_SIZE,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    crop_pct=data_config['crop_pct'])

# Define representative dataset generator
def get_representative_dataset(n_iter: int, dataset_loader: Iterator[Tuple]):
    """
    This function creates a representative dataset generator. The generator yields numpy
        arrays of batches of shape: [Batch, H, W ,C].
    Args:
        n_iter: number of iterations for MCT to calibrate on
    Returns:
        A representative dataset generator
    """       
    def representative_dataset() -> Iterator[List]:
        ds_iter = iter(dataset_loader)
        for _ in range(n_iter):
            yield [next(ds_iter)[0]]

    return representative_dataset

# Get representative dataset generator
representative_dataset_gen = get_representative_dataset(n_iter=n_iters,
                                                        dataset_loader=representative_dataset)

# Set IMX500-v1 TPC
tpc = mct.get_target_platform_capabilities(fw_name="pytorch",
                                           target_platform_name='imx500',
                                           target_platform_version='v1')

# Specify the necessary configuration for mixed precision quantization. To keep the tutorial brief, we'll use a small set of images and omit the hessian metric for mixed precision calculations. It's important to be aware that this choice may impact the resulting accuracy. 
mp_config = mct.core.MixedPrecisionQuantizationConfig(num_of_images=5,
                                                      use_hessian_based_scores=False)
config = mct.core.CoreConfig(mixed_precision_config=mp_config,
                             quantization_config=mct.core.QuantizationConfig(shift_negative_activation_correction=True))

# Define target Resource Utilization for mixed precision weights quantization (75% of 'standard' 8bits quantization)
resource_utilization_data = mct.core.pytorch_resource_utilization_data(in_model=model,
                                                                       representative_data_gen=
                                                                       representative_dataset_gen,
                                                                       core_config=config,
                                                                       target_platform_capabilities=tpc)
resource_utilization = mct.core.ResourceUtilization(weights_memory=resource_utilization_data.weights_memory * 0.75)

# Perform post training quantization
quant_model, _ = mct.ptq.pytorch_post_training_quantization(in_module=model,
                                                            representative_data_gen=
                                                            representative_dataset_gen,
                                                            target_resource_utilization=resource_utilization,
                                                            core_config=config,
                                                            target_platform_capabilities=tpc)
print('Quantized model is ready')

### Model Export

Now, we can export the quantized model, ready for deployment, into a `.onnx` format file. Please ensure that the `save_model_path` has been set correctly. 

In [ ]:
mct.exporter.pytorch_export_model(model=quant_model,
                                  save_model_path='./qmodel.onnx',
                                  repr_dataset=representative_dataset_gen)

### Gradient-Based Post Training Quantization using Model Compression Toolkit
Here we demonstrate how to further optimize the quantized model performance using gradient-based PTQ technique.
**Please note that this section is computationally heavy, and it's recommended to run it on a GPU. For fast deployment, you may choose to skip this step.** 

We will start by loading the COCO training set, and re-define the representative dataset accordingly. 

In [ ]:
!mkdir imagenet/train
!wget https://image-net.org/data/ILSVRC/2012/ILSVRC2012_img_train.tar
!mv ILSVRC2012_img_train.tar imagenet/
!tar -xvf imagenet/ILSVRC2012_img_train.tar -C imagenet/train
!echo Done loading ILSVRC2012_img_train images

GPTQ_REPRESENTATIVE_DATASET_FOLDER = './imagenet/train'
BATCH_SIZE = 4
n_iters = 20

# Load representative dataset
data_config = resolve_data_config(args=DATA_ARGS,
                                  model=model)

train_dataset = create_dataset(name=DATASET, root=DATASET_DIR, is_training=False,
                               batch_size=BATCH_SIZE)
gptq_representative_dataset = create_loader(
    train_dataset,
    input_size=data_config['input_size'],
    batch_size=BATCH_SIZE,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    crop_pct=data_config['crop_pct'])

# Get representative dataset generator
gptq_representative_dataset_gen = get_representative_dataset(n_iter=n_iters,
                                                        dataset_loader=gptq_representative_dataset)

Next, we'll set up the Gradient-Based PTQ configuration and execute the necessary MCT command. Keep in mind that this step can be time-consuming, depending on your runtime.

In [ ]:
# Specify the necessary configuration for Gradient-Based PTQ.
n_gptq_epochs = 1000
gptq_config = mct.gptq.get_pytorch_gptq_config(n_epochs=n_gptq_epochs, use_hessian_based_weights=False)

# Perform Gradient-Based Post Training Quantization
gptq_quant_model, _ = mct.gptq.pytorch_gradient_post_training_quantization(
    model=model,
    representative_data_gen=gptq_representative_dataset_gen,
    target_resource_utilization=resource_utilization,
    gptq_config=gptq_config,
    core_config=config,
    target_platform_capabilities=tpc)

print('Quantized-GPTQ model is ready')

### Model Export

Now, we can export the quantized model, ready for deployment, into a `.onnx` format file. Please ensure that the `save_model_path` has been set correctly. 

In [ ]:
mct.exporter.pytorch_export_model(model=gptq_quant_model,
                                  save_model_path='./qmodel_gptq.onnx',
                                  repr_dataset=gptq_representative_dataset_gen)

## Evaluation on ImageNet dataset

### Floating point model evaluation
Please ensure that the dataset path has been set correctly before running this code cell.

In [ ]:
from tutorials.quick_start.pytorch_fw.utils import classification_eval

EVAL_DATASET = 'Imagenet'
EVAL_DATASET_FOLDER = './imagenet/val'
IMG_SIZE = 256
DATA_ARGS = {'img_size': IMG_SIZE}

# Load representative dataset
data_config = resolve_data_config(args=DATA_ARGS,
                                  model=model)

val_dataset = create_dataset(name=DATASET, root=EVAL_DATASET_FOLDER, is_training=False,
                             batch_size=BATCH_SIZE)
val_loader = create_loader(
    val_dataset,
    input_size=data_config['input_size'],
    batch_size=BATCH_SIZE,
    interpolation=data_config['interpolation'],
    mean=data_config['mean'],
    std=data_config['std'],
    crop_pct=data_config['crop_pct'])

# Evaluate the model on ImageNet
eval_results = classification_eval(model, val_loader)

# Print float model Accuracy results
print("Float model Accuracy: {:.4f}".format(round(100 * eval_results[0], 2)))

### Quantized model evaluation
We can evaluate the performance of the quantized model. There is a slight decrease in performance that can be further mitigated by either expanding the representative dataset or employing MCT's advanced quantization methods, such as GPTQ (Gradient-Based/Enhanced Post Training Quantization).

In [ ]:
# Evaluate the quantized model on ImageNet
eval_results = classification_eval(quant_model, val_loader)

# Print quantized model Accuracy results
print("Quantized model Accuracy: {:.4f}".format(round(100 * eval_results[0], 2)))

Finally, we can evaluate the performance of the quantized model through GPTQ (Gradient-Based/Enhanced Post Training Quantization). We anticipate an improvement in performance compare to the quantized model utilizing PTQ.

In [ ]:
# Evaluate the quantized using GPTQ model on ImageNet
eval_results = classification_eval(gptq_quant_model, val_loader)

# Print quantized using GPTQ model Accuracy results
print("Quantized using GPTQ model Accuracy: {:.4f}".format(round(100 * eval_results[0], 2)))

\
Copyright 2024 Sony Semiconductor Israel, Inc. All rights reserved.

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.